In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
import numpy as np
wine_data=pd.read_csv("/content/drive/MyDrive/Colab Notebooks/data/X.csv")
wine_target=pd.read_csv("/content/drive/MyDrive/Colab Notebooks/data/T.csv")

wine_features=["fixed acidity","volatile acidity","citric acid","residual sugar","chlorides","free sulfur dioxide","total sulfur dioxide","density","pH","sulphates","alcohol"]
X=wine_data[wine_features]
Y=wine_target

After observing the distribution of each feature with their corresponding target.(by scatter plot)  
and regression result with each single feature  
I found almost every features have a linear-like distribution,  
Thus, I choose polynomial functions as the basis funcions for my regression process 

**----------------------Below is the Function define stage-------------------------**

In [ ]:
def split_DataFrame(X,N): #Can't handle Series
    df_1=X.iloc[:N,:]
    df_2=X.iloc[N:,:]
    return df_1, df_2
def variance_numpy(W0,W,X,Y):
    N,D=X.shape
    count=0
    for i in range(N):
        count+=(predict(W0, W, X[i])-Y[i])**2
    count/=N
    return count
def gaussian(x,u,var):
    deviation=np.sqrt(var)
    tmp=np.sqrt(2*np.pi)
    probability=np.exp(-(x-u)**2/(2*var))/(deviation*tmp)
    return probability
def predict(W0,W1,X):
    X=X.reshape(len(X),1)
    return np.matmul(W1,X)+W0
def regression_M(X,Y,M):
    X_numpy=X.to_numpy()
    H=X_numpy
    N, D=X_numpy.shape
    i=0
    for m in range(1,M):
        coefs=H.size/N
        while i<coefs :
            for j in range(D):
                tmp=H[:,i]*X_numpy[:,j]
                tmp=tmp.reshape(N,1)
                H=np.append(H,tmp,1)
                #print(H.shape)
            i+=1
    
    ones=np.array([[1]]*N)
    H=np.append(H,ones,1)
    Y_numpy=Y.to_numpy()
    H_trans=np.transpose(H)
    mul=np.matmul(H_trans,H)
    tmp=np.linalg.pinv(mul)
    W=np.matmul(np.matmul(tmp,H_trans),Y_numpy)
    intercept=W[-1]
    W=np.delete(W,-1)
    H=np.delete(H,-1,1)
    return intercept, W, H
def polynomial_form_M(X, M):
    X_numpy=X.to_numpy()
    H=X_numpy
    N, D=X_numpy.shape
    i=0
    for m in range(1,M):
        coefs=H.size/N
        while i<coefs :
            for j in range(D):
                tmp=H[:,i]*X_numpy[:,j]
                tmp=tmp.reshape(N,1)
                H=np.append(H,tmp,1)
                #print(H.shape)
            i+=1
    return H

Below are the maximize likelihood process with different order of polynomial regression.  
Because the probability is to low to show even for float64,  
So here I introduce log likelihood with base e


In [ ]:
X_train, X_test=split_DataFrame(X,1500)
Y_train, Y_test=split_DataFrame(Y,1500)
X_train_numpy=X_train.to_numpy()
Y_train_numpy=Y_train.to_numpy()
X_test_numpy=X_test.to_numpy()
Y_test_numpy=Y_test.to_numpy()


for m in range(1,4):
  W0, W, H_train=regression_M(X_train,Y_train,m)
  var=variance_numpy(W0, W, H_train, Y_train_numpy)
  log_likelihood=np.float64(0)
  
#--------Testing stage------------ 
  H_test=polynomial_form_M(X_test,m)
  N,D=H_test.shape
  for i in range(N):
      p=gaussian(Y_test_numpy[i],predict(W0,W,H_test[i]),var)
      log_likelihood+=np.log(p)
  print("M=",m, "avg log_likelihood of test",log_likelihood/N)
  log_likelihood=np.float64(0)
  for i in range(N):
      p=gaussian(Y_train_numpy[i],predict(W0,W,H_train[i]),var)
      log_likelihood+=np.log(p)
  print("M=",m, "avg log_likelihood of train(first 99))",log_likelihood/N)
  print("----------------------------------------")

M= 1 avg log_likelihood of test [-0.89451762]
M= 1 avg log_likelihood of train(first 99)) [-0.99533061]
----------------------------------------
M= 2 avg log_likelihood of test [-1.06810252]
M= 2 avg log_likelihood of train(first 99)) [-0.8851133]
----------------------------------------
M= 3 avg log_likelihood of test [-1.85180558]
M= 3 avg log_likelihood of train(first 99)) [-0.78883154]
----------------------------------------


The result above shows that although the training likelihood of the higher order perform well  
but the likelihood of test decrease sharply  
over-fitting problem seems to occur when we try to increase the order of polynomial regression

Below I try to do N-cross validation to obtain more precise outcome  

for deciding which order of regression is more suitable for maximize likelihood without over-fitting
  
Here I changing hyperparameter(order)

In [ ]:
def Validation_MLE_poly(X_train,X_test,Y_train,Y_test,order):
  X_train_numpy=X_train.to_numpy()
  Y_train_numpy=Y_train.to_numpy()
  X_test_numpy=X_test.to_numpy()
  Y_test_numpy=Y_test.to_numpy()
  ret=[]

  for m in range(1,order+1):
    W0, W, H_train=regression_M(X_train,Y_train,m)
    var=variance_numpy(W0, W, H_train, Y_train_numpy)
    
#--------Testing stage------------ 
    log_likelihood=np.float64(0)
    H_test=polynomial_form_M(X_test,m)
    N,D=H_test.shape
    for i in range(N):
      p=gaussian(Y_test_numpy[i],predict(W0,W,H_test[i]),var)
      log_likelihood+=np.log(p)
    log_likelihood/=N
    print("M=",m, "avg log_likelihood",log_likelihood)
    ret.append(log_likelihood)
 
  return ret

In [ ]:
def N_set(data,cut_size):
  data_set=[]
  N,D=data.shape
  cur=0
  while cur<(N-cut_size):
    data_set.append(data.iloc[cur:cur+cut_size,:])
    cur+=cut_size
    
  data_set.append(data.iloc[cur:,:])
  return data_set

After defining the function for N_cross validation  

Below is the N_cross validation of each subset have the size=100  
we execute the polynomial regression only for M=1~3  
because M=4 is too large to handle(16105 basis)

In [ ]:
cut_size=100
Max_poly_order=3

X_sets=N_set(X,cut_size)
Y_sets=N_set(Y,cut_size)
times=len(X_sets)

count=[np.float64(0)]*Max_poly_order
for t in range(times):
  print("----------------time=",t,"-------------------------")
  X_train=pd.DataFrame()
  Y_train=pd.DataFrame()
  X_test=X_sets[t].copy()
  Y_test=Y_sets[t].copy()
  for i in range(times):
    if i==t :
      continue
    else:
      X_train=X_train.append(X_sets[i])
      Y_train=Y_train.append(Y_sets[i])
  tmp=Validation_MLE_poly(X_train, X_test, Y_train, Y_test, Max_poly_order)
  for i in range(Max_poly_order):
    count[i]+=tmp[i]
print("############################################")
for i in range(Max_poly_order):
  print("average cross validation likelihood of M=",i+1,": ",count[i]/times)

----------------time= 0 -------------------------
M= 1 avg log_likelihood [-1.07303505]
M= 2 avg log_likelihood [-1.05553961]
M= 3 avg log_likelihood [-5.20461925]
----------------time= 1 -------------------------
M= 1 avg log_likelihood [-0.87676809]
M= 2 avg log_likelihood [-0.90664619]
M= 3 avg log_likelihood [-0.93516671]
----------------time= 2 -------------------------
M= 1 avg log_likelihood [-1.02347907]
M= 2 avg log_likelihood [-1.16662856]
M= 3 avg log_likelihood [-1.00146888]
----------------time= 3 -------------------------
M= 1 avg log_likelihood [-0.98253852]
M= 2 avg log_likelihood [-1.00515208]
M= 3 avg log_likelihood [-1.15430724]
----------------time= 4 -------------------------
M= 1 avg log_likelihood [-1.15715304]
M= 2 avg log_likelihood [-1.22698465]
M= 3 avg log_likelihood [-1.31995545]
----------------time= 5 -------------------------
M= 1 avg log_likelihood [-0.95007759]
M= 2 avg log_likelihood [-0.97082579]
M= 3 avg log_likelihood [-0.90563096]
----------------

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:18: RuntimeWarning: divide by zero encountered in log


M= 3 avg log_likelihood [-inf]
----------------time= 15 -------------------------
M= 1 avg log_likelihood [-0.89451762]
M= 2 avg log_likelihood [-1.06810252]
M= 3 avg log_likelihood [-1.85180558]
############################################
average cross validation likelihood of M= 1 :  [-1.00224412]
average cross validation likelihood of M= 2 :  [-1.04666835]
average cross validation likelihood of M= 3 :  [-inf]


After observing the result above

I found that the over-fitting condition do occur for higher order polynomial regression like the "-inf" in time=14 M=3  

So I conclude that M=1 order regression is the most suitable polynomial regression model for this problem